In [1]:
import imageio
import pathlib
import numpy as np

In [2]:
testpath = pathlib.Path('.')
list(testpath.glob('**/*.ipynb'))

[WindowsPath('Necleus.detection.imageio.ipynb'),
 WindowsPath('Necleus.detection.tensorflow.ipynb'),
 WindowsPath('.ipynb_checkpoints/Necleus.detection.imageio-checkpoint.ipynb'),
 WindowsPath('.ipynb_checkpoints/Necleus.detection.tensorflow-checkpoint.ipynb'),
 WindowsPath('input/test.ipynb'),
 WindowsPath('input/.ipynb_checkpoints/test-checkpoint.ipynb')]

In [3]:
training_path = pathlib.Path('input/stage1_train').glob('**/*.png')
training_sorted = sorted([x for x in training_path])
im_path = training_sorted[45]
im = imageio.imread(str(im_path))

In [ ]:
#im_path = training_sorted[45]
#im = imageio.imread(str(im_path))
type(im)
type(im_path)

In [4]:
# RGB to grayscale
# print('Original image shape:{}'.format(im.shape))
from skimage.color import rgb2gray
im_gray = rgb2gray(im)
# print('New image shape:{}'.format(im.shape))

In [5]:
from skimage.filters import threshold_otsu

In [6]:
thresh_val = threshold_otsu(im_gray)

In [7]:
# create mask based on the thresh value
mask = np.where(im_gray > thresh_val, 1, 0)
# Make sure that larger portion of the mask is background
if np.sum(mask==0) < np.sum(mask==1):
    mask = np.where(mask, 0, 1)
# np.where returns ndarray or tuple of ndarrays
# If both x and y are specified, the output array contains elements of x where condition is True, and elements from y elsewhere.

In [8]:
from scipy import ndimage

In [9]:
labels, nlabels = ndimage.label(mask)
label_arrays = []
print(nlabels)
for label_num in range(1, nlabels+1):
    label_mask = np.where(labels==label_num, 1, 0)
    label_arrays.append(label_mask)
    
print("{} object detected".format(nlabels))

1
1 object detected


In [10]:
for label_ind, label_coords in enumerate(ndimage.find_objects(labels)):
    cell = im_gray[label_coords]

# Check if the label size is too small
if np.product(cell.shape) < 10:
    print('Label {} is too small! Setting to 0.'.format(label_ind))
    mask = np.where(labels==label_ind+1, 0, mask)
    
# Regenerate the labels
labels, nlabels = ndimage.label(mask)
print('There are now {} objects detected'.format(nlabels))

There are now 1 objects detected


In [15]:
two_cell_indices = ndimage.find_objects(labels)[0]
cell_mask = mask[two_cell_indices]
cell_mask_opened = ndimage.binary_opening(cell_mask, iterations=8)
cell_mask

array([[0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0],
       [0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0],
       [0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0]])